# Steps for the execution of the program: 
 1. Importing the modules.
 2. Creation of the descriptive table.
 
   2.1. Reading the names of files.
   
   2.2. Implementation of the function fill_table.
   
   2.3. Implementation of the function preproc.
   
   2.4. Delete the variables with a determined percent of missing values.

###  1. Importing the modules.

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

### 2. Creation of the descriptive table.

#### 2.1. Reading the names of files.

In [20]:
mypath = "C:/Users/miguelico/Google Drive/master/TFM/Data/Variables"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = [f.replace('.csv', '') for f in onlyfiles]

In [22]:
onlyfiles

['About',
 'AcuteHealth',
 'Brief_Motor_Screen',
 'Cannabis_Use_in_PD',
 'Clinical_Global_Impression_of_Change_Non_PD',
 'Clinical_Global_Impression_of_Change_PD',
 'Compensation_Strategies',
 'COVID_19_Experience',
 'CurrentHealth',
 'DailyActivity',
 'DailyLiving',
 'EEQ_alcohol',
 'EEQ_Anti_Inflammatory_Medication_History',
 'EEQ_caffeine',
 'EEQ_calcium_channel_blocker_medication_history',
 'EEQ_female_reproductive_health',
 'EEQ_Head_Injury_or_Concussion',
 'EEQ_height_and_weight',
 'EEQ_occupation',
 'EEQ_pesticides_at_work',
 'EEQ_physical_activity',
 'EEQ_residential',
 'EEQ_smoking',
 'EEQ_toxicant',
 'Episodes',
 'Experiences_with_Sensory_Misperceptions',
 'Financial_and_Social_Impact_of_Parkinsons_Disease',
 'FIVE',
 'General',
 'Genetic',
 'Handedness',
 'HealthHis',
 'Impact_and_Communication_about_OFF_Periods',
 'MedHis',
 'Medications',
 'MedicationsPD',
 'Mood',
 'Movement',
 'NeurHist',
 'NeuroHis2',
 'NonMovement',
 'PASE',
 'Patient_Therapeutic_Preferences_Questionna

#### 2.2. Implementation of the function fill_table.
This function provides a DataFrame with the most important aspect of our dataset. Here, you can see the list of variables:

 - **Variable**: This column represents the name of the column.
 - **Number of rows**: This column represent the umber of rows of the column. The number is the same for all variables of a same table.
 - **Number of different id**: This variable represents the number of different's ids that Variable has in a table.
 - **Number of NaN**: This variable represents the number of the missing values which there are in each column.
 - **Number of different values**: This column represents the number of different values which are in each column.
 - **Number of PD**: This column represents the number of Patients with Parkinson (PwP), we calculate this number by using the fox_insight_id and the table Users.
 - **Number of EarlyPD**: This column represents the number of PwP who have the disease for a period less than 3 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Number of MidPD**: This column represents the number of PwP who have the disease for a period between 3 years and 10 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Number of LatePD**: This column represents the number of PwP who have the disease for a period more than 10 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Name of table**: This column represents the name of table where the column belongs.
 - **dtype**: This column represents the type of variable.


In [8]:
def fill_table(field):

#Importación de las distintas tablas en formato .csv y conversión a un DataFrame para su manipulación.
    
    root = "C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/" + field + ".csv"
    Users = pd.read_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables/Users.csv" , sep = ",")
    df1 = pd.read_csv(root , sep = ",")
    
    df = pd.DataFrame(columns=['Variable', 'Number of rows', 'Number of different id', 'Number of NaN', 'Number of different values', 'Number of PD', 'Number of EarlyPD', 
                               'Number of MidPD', 'Number of LatePD', 'Name of table', 'dtype'])
    df['Variable'] = list(df1.columns)
    df['Number of rows'] = df1.shape[0]
    df['Number of different id'] = len(df1['fox_insight_id'].unique())
    num_nan = []
    [num_nan.append(df1[x].isna().sum()) for x in list(df1.columns)]
    df['Number of NaN'] = num_nan
    df['Number of different values'] = [len(df1[x][df1[x].notna()].unique()) for x in df1.columns]
    df['Name of table'] = field
    df['dtype'] = [x for x in df1.dtypes.to_frame().reset_index()[0]]
    if field != 'Users':
        df1 = df1.merge(Users[['fox_insight_id', 'InitPDDiag', 'YearsWithPD']], how = 'inner', on = 'fox_insight_id')
    df['Number of PD'] = len(df1['fox_insight_id'][df1['InitPDDiag'] == 1].unique())
    df['Number of EarlyPD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 0].unique())
    df['Number of MidPD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 1].unique())
    df['Number of LatePD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 2].unique())
    
    df = df.drop_duplicates()
    return df

In [10]:
table2 = pd.DataFrame(columns = ['Variable', 'Number of rows', 'Number of different id', 'Number of NaN', 'Number of different values', 'Number of PD', 'Number of EarlyPD', 
                               'Number of MidPD', 'Number of LatePD', 'Name of table', 'dtype'] )
for x in onlyfiles:
    table2 = table2.append(fill_table(x))

In [11]:
table2

,Variable,Number of rows,Number of different id,Number of NaN,Number of different values,Number of PD,Number of EarlyPD,Number of MidPD,Number of LatePD,Name of table,dtype
0,fox_insight_id,177143,50565,0,50565,36455,16392,13868,5917,About,object
1,age,177143,50565,0,864,36455,16392,13868,5917,About,float64
2,days_elapsed,177143,50565,0,23239,36455,16392,13868,5917,About,int64
3,days_acquired,177143,50565,0,31,36455,16392,13868,5917,About,int64
4,schedule_of_activities,177143,50565,1796,47,36455,16392,13868,5917,About,object
...,...,...,...,...,...,...,...,...,...,...,...
21,DiagWhoNeurPD,50607,50607,35,2,36485,16393,13868,5917,Users,float64
22,DiagWhoSpecPD,50607,50607,35,2,36485,16393,13868,5917,Users,float64
23,DiagWhoOtherPD,50607,50607,35,2,36485,16393,13868,5917,Users,float64
24,InitReferralSource,50607,50607,5862,12,36485,16393,13868,5917,Users,float64


#### 2.3. Implementation of the function preproc.

In [13]:
def preproc(table, folder):
    field = 'FoxInsightVariables (1)'
    root = "C:/Users/miguelico/Google Drive/master/TFM/Data/" + field + ".csv"
    Variables = pd.read_csv(root , sep = ",")
    Variables = Variables.rename(columns = {'variable': 'Variable'})
    table = table.merge(Variables, how = 'inner', on = 'Variable')
    table = table.drop_duplicates()
    table['Longitudinal'] = ["No" if table['Number of rows'].iloc[i] == table['Number of different id'].iloc[i] else 'Yes' for i in range(table.shape[0])]
    table = table.drop(table[(table['Name of table'] != 'Users') & ((table['Variable'] == 'InitPDDiag') | (table['Variable'] == 'YearsWithPD'))].index)      
    table.to_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/" + folder + "/Análisis descriptivo y explotario de las variables.csv", index=False)
    return table

In [ ]:
AD = preproc(table2, 'Variables limpiadas')

#### 2.4. Delete the variables with a determined percent of missing values.

In [14]:
def delete_columns(perc):
    #Importamos la tabla que queremos borrar 
    for y in AD['Name of table'].unique():
        cond1 = AD['Name of table'] == y
        root = "C:/Users/miguelico/Google Drive/master/TFM/Data/Variables/" + y + ".csv"
        df = pd.read_csv(root , sep = ",")
        for x in list(AD['Variable'][cond1].unique()):
            cond2 = AD['Variable'] == x
            if float(AD['Number of NaN'][cond1 & cond2]/AD['Number of rows'][cond1 & cond2])>=0.9:
                df = df.drop(x, axis = 1)
        df.to_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/"+ y + ".csv", index=False)